In [20]:
import pandas as pd 
import numpy as np

orders = pd.read_csv('OutputData.csv')
orders.head()

od_mat = pd.read_csv('./data/Sample_OD_Matrix.csv', index_col= 0)
od_mat.head()

rack_capa = 2
racks = od_mat.index[2:]

In [25]:
sku_freq = orders['SKU_CD'].value_counts() # sku_cd 10
sku_freq.head()

cart_skus = orders.groupby('CART_NO')['SKU_CD'].apply(list).to_dict()
cart_skus # cart_no : [sku_list]

sku_seq = orders.groupby('SKU_CD')['SEQ'].min().to_dict()
sku_seq # sku_cd : min(seq)

assigned_sku = set()
rack_assign_count = {rack: 0 for rack in racks}
new_sku_to_loc = {}

# 대표 SKU 후보
cart_main_sku = {}
for cart, sku_list in cart_skus.items() :
  sku_list = list(set(sku_list))
  # 대표 SKU : 빈도 높은순 / 앞 순서 순
  sku_list.sort(key= lambda x : (-sku_freq.get(x, 0), sku_seq.get(x, float('inf'))))
  cart_main_sku[cart] = sku_list[0]

cart_main_sku # 'CART_NO : main_SKU
# 대표 SKU 우선순위(-빈도, 앞 순서)로 CART 정렬
sorted_cart_by_main_sku = sorted(cart_main_sku.items(),
                                 key= lambda x : (-sku_freq.get(x[1], 0), sku_seq.get(x[1], float('inf'))))
sorted_cart_by_main_sku # (빈도 높고 앞 순서인 대표 SKU를 포함한 카트)

for cart, main_sku in sorted_cart_by_main_sku :
  cart_df = orders[orders['CART_NO'] == cart].copy()
  cart_df = cart_df.sort_values('SEQ')
  cart_skus = cart_df['SKU_CD'].unique().tolist()
  
  if main_sku not in assigned_sku :
    for rack in od_mat.loc[od_mat.index[0], racks].sort_values().index :
      if rack_assign_count[rack] < rack_capa :
        new_sku_to_loc[main_sku] = rack 
        assigned_sku.add(main_sku)
        rack_assign_count[rack] += 1 
        base_rack = rack 
        break 
  else :
    base_rack = new_sku_to_loc[main_sku]
  
  remaining_skus = [sku for sku in cart_skus if sku!= main_sku and sku not in assigned_sku]
  # 빈도순 / 앞 순서순
  remaining_skus.sort(key= lambda x: (-sku_freq.get(x, 0), sku_seq.get(x, float('inf'))))
  
  for sku in remaining_skus :
    nearby_racks = od_mat.loc[base_rack, racks].sort_values().index
    for rack in nearby_racks :
      if rack_assign_count[rack] < rack_capa :
        new_sku_to_loc[sku] = rack 
        assigned_sku.add(sku)
        rack_assign_count[rack] += 1 
        break 
      
orders['LOC'] = orders['SKU_CD'].map(new_sku_to_loc)
print("✅ 미배치 SKU 수:", len(set(orders['SKU_CD']) - set(new_sku_to_loc)))
print("✅ 남은 빈 랙 수:", sum(v < rack_capa for v in rack_assign_count.values()))


✅ 미배치 SKU 수: 0
✅ 남은 빈 랙 수: 0
